# Import des données depuis la page de recherche regroupant tous les tests

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Récupération des liens amenant à chaque fiche jeu

In [6]:
links=[]

#Boucle pour récupérer les liens dans les pages de listing des tests de jeuxvideo.com
for x in range(1,933): 
    url="https://www.jeuxvideo.com/tests/?p=" 
    r=requests.get(url+str(x)) #Il y a 932 pages de tests, x vient complèter l'url pour scraper toutes les pages
    soup=BeautifulSoup(r.content,"html.parser")
    content=soup.find_all("div",class_="container__3zUztO iconPlayHover") #Récupération du bloc principal de la page

    #Boucle pour récupérer, parmi les éléments du bloc principal de la page scrapée, 
    #les extensions des liens amenant vers la page test de chaque jeu
    for property in content:
        link=property.find('a', {"class": "bonusLabel__3bIEJW gamePageLink__25_ek7"}, href=True)
        if link!=None:
            links.append(link["href"])
        
print(links)

['/jeux/jeu-1389738/', '/jeux/jeu-1422472/', '/jeux/jeu-860674/', '/jeux/jeu-1422394/', '/jeux/jeu-1528977/', '/jeux/switch/jeu-1365387/', '/jeux/jeu-1388525/', '/jeux/jeu-1237529/', '/jeux/switch/jeu-1468296/', '/jeux/pc/jeu-1358416/', '/jeux/jeu-1525460/', '/jeux/jeu-79026/', '/jeux/jeu-1441343/', '/jeux/jeu-1117583/', '/jeux/jeu-1056358/', '/jeux/jeu-1087099/', '/jeux/pc/jeu-1151453/', '/jeux/jeu-1419463/', '/jeux/jeu-1237618/', '/jeux/jeu-463006/', '/jeux/pc/00054835-lost-ark.htm', '/jeux/jeu-1369187/', '/jeux/jeu-1511222/', '/jeux/jeu-1391906/', '/jeux/jeu-859486/', '/jeux/jeu-1222521/', '/jeux/pc/jeu-1519330/', '/jeux/jeu-1459522/', '/jeux/switch/jeu-1370636/', '/jeux/jeu-903696/', '/jeux/jeu-1514874/', '/jeux/jeu-1439945/', '/jeux/jeu-1058108/', '/jeux/jeu-1381247/', '/jeux/jeu-446727/', '/jeux/pc/jeu-898287/', '/jeux/jeu-1290320/', '/jeux/pc/jeu-665056/', '/jeux/jeu-410392/', '/jeux/jeu-1189058/', '/jeux/pc/jeu-1340061/', '/jeux/jeu-1192824/', '/jeux/jeu-1511199/', '/jeux/jeu-1

In [3]:
#Affichage du nombre d'extensions de liens récupérés
print(len(links))

15369


# VERSION 1 : import des données depuis chaque fiche jeu via boucle for

In [7]:
#Création d'une liste
games_jv3=[]

#Boucle qui pour chaque lien des pages test récupérées va permettre de chercher le nom du jeu, les plateformes,
#le nombre d'avis par plateforme, la note joueurs par plateforme, la note des journalistes et la date de lancement
for y in links:
        url2 = "https://www.jeuxvideo.com"
        r2 = requests.get(url2 + y) #chaque extention de lien est appelée pour complèter le host name de jeuxvideo.com

        soup2 = BeautifulSoup(r2.content, "html.parser")
        content2 = soup2.find_all("div", class_="layout layout--classic js-layout")
        
        names = soup2.find("h1",class_="breadcrumb__item").text #récupération des noms du jeu vidéo
        
        #récupération des plateformes sur lesquelles sont sorties le jeu    
        plateforms = soup2.find_all("span",class_="userOpinionsPerPlatform__platformLabel") 
        
        #récupération des avis par plateforme    
        reviews = soup2.find_all("span",class_="userOpinionsPerPlatform__ratingCount")
        
        #récupération des notes des joueurs par plateforme
        players_rates = soup2.find_all("span",class_="userOpinionsPerPlatform__averageRating")
        
        #récupération de la note journalistes si existante
        zone_rate_journalist=soup2.find("div",class_="gameCharacteristicsMain__gauge")
        if zone_rate_journalist == None:
            rate_journalist = None
        else:
            rate_journalist = soup2.find("text",class_="gameCharacteristicsMain__gaugeText").text
        
        #récupération de la date de lancement du jeu si présente sur la page
        zone_launch_date = soup2.find("div",class_="gameCharacteristicsMain__releaseDate")
        if zone_launch_date == None:
            launch_date = None
        else:
            launch_date = soup2.find("div",class_="gameCharacteristicsMain__releaseDate").text
        
        #création d'un dictionnaire avec les données récupérées
        property_info3={"names":names,
                        "plateforms":plateforms,
                        "reviews":reviews,
                        "players_rates":players_rates,
                        "journalist_rate":rate_journalist,
                       "launch_date":launch_date}
        
        #Enrichissement de la liste avec le dictionnaire créé
        games_jv3.append(property_info3)

#Création d'un dataframe avec ajustement de la date de lancement        
df=pd.DataFrame(games_jv3)
df["launch_date"]=df["launch_date"].apply(lambda x: x[12:]).str.strip()
df

,names,plateforms,reviews,players_rates,journalist_rate,launch_date
0,WWE 2K22,"[[PC], [PS5], [Xbox Series], [PS4], [ONE]]","[[(4 avis)], [(0 avis)], [(0 avis)], [(0 avis)...",[[15]],15,11 mars 2022
1,FAR : Changing Tides,"[[PC], [PS5], [Xbox Series], [Switch], [PS4], ...","[[(3 avis)], [(0 avis)], [(1 avis)], [(0 avis)...","[[17], [16]]",14,01 mars 2022
2,Babylon's Fall,"[[PC], [PS5], [PS4]]","[[(0 avis)], [(1 avis)], [(3 avis)]]","[[0], [9]]",12,03 mars 2022
3,Final Fantasy Pixel Remaster,"[[PC], [iOS], [Android]]","[[(10 avis)], [(1 avis)], [(1 avis)]]","[[6.3], [10], [10]]",18,29 juil. 2021
4,Assassin's Creed Valhalla : Dawn of Ragnarök,"[[PC], [PS5], [Xbox Series], [PS4], [ONE], [St...","[[(3 avis)], [(1 avis)], [(0 avis)], [(0 avis)...","[[12], [0]]",15,10 mars 2022
...,...,...,...,...,...,...
1907,Mr. Robot eps1.51exfiltrati0n,"[[iOS], [Android]]","[[(0 avis)], [(1 avis)]]",[[13]],14,16 août 2016
1908,FIFA 17,"[[PC], [PS4], [ONE], [PS3], [360]]","[[(208 avis)], [(209 avis)], [(72 avis)], [(8 ...","[[8.4], [10], [12], [7.4], [8.7]]",16,29 sept. 2016
1909,NBA 2K17,"[[PC], [PS4], [ONE], [PS3], [360], [iOS], [And...","[[(32 avis)], [(27 avis)], [(13 avis)], [(1 av...","[[13], [15.3], [14.5], [14]]",17,16 sept. 2016
1910,Obduction,"[[PC], [PS4], [ONE], [Mac]]","[[(13 avis)], [(1 avis)], [(0 avis)], [(0 avis)]]","[[14.9], [16]]",13,24 août 2016


# VERSION 2 : import des données depuis chaque fiche jeu via déf fonction

In [8]:
#Fonction qui prend en paramètre l'url d'une page test de jeu vidéo et qui permet de retourner le nom du jeu, les plateformes,
#le nombre d'avis par plateformes, la note joueurs par plateforme, la note des journalistique et la date de lancement
def extract_data(url_test):
        names = url_test.find("h1",class_="breadcrumb__item").text
        
            
        plateforms = url_test.find_all("span",class_="userOpinionsPerPlatform__platformLabel")
        
            
        reviews = url_test.find_all("span",class_="userOpinionsPerPlatform__ratingCount")
        
        
        players_rates = url_test.find_all("span",class_="userOpinionsPerPlatform__averageRating")
        
        zone_rate_journalist = url_test.find("div",class_="gameCharacteristicsMain__gauge")
        if zone_rate_journalist == None:
            rate_journalist = None
        else:
            rate_journalist = url_test.find("text",class_="gameCharacteristicsMain__gaugeText").text
            
        zone_launch_date = url_test.find("div",class_="gameCharacteristicsMain__releaseDate")
        if zone_launch_date == None:
            launch_date = None
        else:
            launch_date = url_test.find("div",class_="gameCharacteristicsMain__releaseDate").text
    
        property_info3={"names":names,
                        "plateforms":plateforms,
                        "reviews":reviews,
                        "players_rates":players_rates,
                        "journalist_rate":rate_journalist,
                       "launch_date":launch_date}
    
        return property_info3

#Création d'une liste
data_full = []
#Création d'une boucle qui va parcourir toutes les urls des pages de test récupérées sur jeuxvideo.com
for url in links:
    url_scrap = "https://www.jeuxvideo.com"+url
    url_page = requests.get(url_scrap)
    parsed_url_page = BeautifulSoup(url_page.content, 'html.parser')
    #Alimentation de la liste avec les résultats obtenus de la fonction extract_data pour chaque url
    data_full.append(extract_data(parsed_url_page)) 

#Création d'un dataframe
df=pd.DataFrame.from_dict(data_full)
df

,names,plateforms,reviews,players_rates,journalist_rate,launch_date
0,WWE 2K22,"[[PC], [PS5], [Xbox Series], [PS4], [ONE]]","[[(4 avis)], [(0 avis)], [(0 avis)], [(0 avis)...",[[15]],15,\nSortie : \n 11 mars 2022\...
1,FAR : Changing Tides,"[[PC], [PS5], [Xbox Series], [Switch], [PS4], ...","[[(3 avis)], [(0 avis)], [(1 avis)], [(0 avis)...","[[17], [16]]",14,\nSortie : \n 01 mars 2022\...
2,Babylon's Fall,"[[PC], [PS5], [PS4]]","[[(0 avis)], [(1 avis)], [(3 avis)]]","[[0], [9]]",12,\nSortie : \n 03 mars 2022\...
3,Final Fantasy Pixel Remaster,"[[PC], [iOS], [Android]]","[[(10 avis)], [(1 avis)], [(1 avis)]]","[[6.3], [10], [10]]",18,\nSortie : \n 29 juil. 2021...
4,Assassin's Creed Valhalla : Dawn of Ragnarök,"[[PC], [PS5], [Xbox Series], [PS4], [ONE], [St...","[[(3 avis)], [(1 avis)], [(0 avis)], [(0 avis)...","[[12], [0]]",15,\nSortie : \n 10 mars 2022\...
...,...,...,...,...,...,...
1907,Mr. Robot eps1.51exfiltrati0n,"[[iOS], [Android]]","[[(0 avis)], [(1 avis)]]",[[13]],14,\nSortie : \n 16 août 2016\...
1908,FIFA 17,"[[PC], [PS4], [ONE], [PS3], [360]]","[[(208 avis)], [(209 avis)], [(72 avis)], [(8 ...","[[8.4], [10], [12], [7.4], [8.7]]",16,\nSortie : \n 29 sept. 2016...
1909,NBA 2K17,"[[PC], [PS4], [ONE], [PS3], [360], [iOS], [And...","[[(32 avis)], [(27 avis)], [(13 avis)], [(1 av...","[[13], [15.3], [14.5], [14]]",17,\nSortie : \n 16 sept. 2016...
1910,Obduction,"[[PC], [PS4], [ONE], [Mac]]","[[(13 avis)], [(1 avis)], [(0 avis)], [(0 avis)]]","[[14.9], [16]]",13,\nSortie : \n 24 août 2016\...


# Nettoyage des colonnes Plateformes, Avis et Notes joueurs

In [9]:
df1=df

#Boucle permettant de supprimer les crochets pour chaque valeur à l'intérieur des listes de plateformes
clean_plateforms=[]
for p in df1["plateforms"]:
    sub_clean=[]
    for x in p:
        sub_clean.append(x.text)
    clean_plateforms.append(sub_clean)
#Création d'une nouvelle colonne avec les valeurs nettoyées
df1["clean_plateforms"]=clean_plateforms

#Boucle permettant de supprimer les crochets pour chaque valeur à l'intérieur des listes d'avis
clean_reviews=[]
for p in df1["reviews"]:
    sub_clean=[]
    for x in p:
        sub_clean.append(x.text)
    clean_reviews.append(sub_clean)
#Création d'une nouvelle colonne avec les valeurs nettoyées    
df1["clean_reviews"]=clean_reviews

#Boucle permettant de supprimer les crochets pour chaque valeur à l'intérieur des listes de notes de joueurs
clean_rates=[]
for p in df1["players_rates"]:
    sub_clean=[]
    for x in p:
        sub_clean.append(x.text)
    clean_rates.append(sub_clean)
#Création d'une nouvelle colonne avec les valeurs nettoyées                
df1["clean_players_rates"]=clean_rates

#Suppression des anciennes colonnes    
df1.pop("plateforms")
df1.pop("reviews")
df1.pop("players_rates")
df1

,names,journalist_rate,launch_date,clean_plateforms,clean_reviews,clean_players_rates
0,WWE 2K22,15,\nSortie : \n 11 mars 2022\...,"[PC, PS5, Xbox Series, PS4, ONE]","[(4 avis), (0 avis), (0 avis), (0 avis), (0 av...",[15]
1,FAR : Changing Tides,14,\nSortie : \n 01 mars 2022\...,"[PC, PS5, Xbox Series, Switch, PS4, ONE]","[(3 avis), (0 avis), (1 avis), (0 avis), (0 av...","[17, 16]"
2,Babylon's Fall,12,\nSortie : \n 03 mars 2022\...,"[PC, PS5, PS4]","[(0 avis), (1 avis), (3 avis)]","[0, 9]"
3,Final Fantasy Pixel Remaster,18,\nSortie : \n 29 juil. 2021...,"[PC, iOS, Android]","[(10 avis), (1 avis), (1 avis)]","[6.3, 10, 10]"
4,Assassin's Creed Valhalla : Dawn of Ragnarök,15,\nSortie : \n 10 mars 2022\...,"[PC, PS5, Xbox Series, PS4, ONE, Stadia]","[(3 avis), (1 avis), (0 avis), (0 avis), (0 av...","[12, 0]"
...,...,...,...,...,...,...
1907,Mr. Robot eps1.51exfiltrati0n,14,\nSortie : \n 16 août 2016\...,"[iOS, Android]","[(0 avis), (1 avis)]",[13]
1908,FIFA 17,16,\nSortie : \n 29 sept. 2016...,"[PC, PS4, ONE, PS3, 360]","[(208 avis), (209 avis), (72 avis), (8 avis), ...","[8.4, 10, 12, 7.4, 8.7]"
1909,NBA 2K17,17,\nSortie : \n 16 sept. 2016...,"[PC, PS4, ONE, PS3, 360, iOS, Android]","[(32 avis), (27 avis), (13 avis), (1 avis), (0...","[13, 15.3, 14.5, 14]"
1910,Obduction,13,\nSortie : \n 24 août 2016\...,"[PC, PS4, ONE, Mac]","[(13 avis), (1 avis), (0 avis), (0 avis)]","[14.9, 16]"


# Remise en ordre des notes par rapport aux avis

In [10]:
#Copie de la nouvelle colonne notes joueurs dans une variable appellée tab
tab=df1["clean_players_rates"]

#Boucle permettant, pour une même ligne, d'avoir autant de notes joueurs que de nombres d'avis
#Une valeur "0 avis" n'avait pas de correspondance note joueur --> ajout de 0 dans les notes joueurs pour avoir le même nombre d'items
for i in range(len(tab)):
    if len(df1["clean_reviews"][i])>len(tab[i]):
        diff=len(df1["clean_reviews"][i])-len(tab[i])
        tab[i].extend(0 for x in range(diff))

#Boucle permettant de créer une nouvelle colonne avec le bon nombre d'items avis/notes joueurs par ligne et une remise à zéro
#Cette colonne sera utile pour remettre les notes joueurs en face du nombre d'avis laissés
tab2=[]
for i in tab:
    sub_tab2=[]
    for j in range(len(i)):
        sub_tab2.append(0)
    tab2.append(sub_tab2)
    
df1["players_rates_ordered"]=tab2
df1

,names,journalist_rate,launch_date,clean_plateforms,clean_reviews,clean_players_rates,players_rates_ordered
0,WWE 2K22,15,\nSortie : \n 11 mars 2022\...,"[PC, PS5, Xbox Series, PS4, ONE]","[(4 avis), (0 avis), (0 avis), (0 avis), (0 av...","[15, 0, 0, 0, 0]","[0, 0, 0, 0, 0]"
1,FAR : Changing Tides,14,\nSortie : \n 01 mars 2022\...,"[PC, PS5, Xbox Series, Switch, PS4, ONE]","[(3 avis), (0 avis), (1 avis), (0 avis), (0 av...","[17, 16, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0]"
2,Babylon's Fall,12,\nSortie : \n 03 mars 2022\...,"[PC, PS5, PS4]","[(0 avis), (1 avis), (3 avis)]","[0, 9, 0]","[0, 0, 0]"
3,Final Fantasy Pixel Remaster,18,\nSortie : \n 29 juil. 2021...,"[PC, iOS, Android]","[(10 avis), (1 avis), (1 avis)]","[6.3, 10, 10]","[0, 0, 0]"
4,Assassin's Creed Valhalla : Dawn of Ragnarök,15,\nSortie : \n 10 mars 2022\...,"[PC, PS5, Xbox Series, PS4, ONE, Stadia]","[(3 avis), (1 avis), (0 avis), (0 avis), (0 av...","[12, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...
1907,Mr. Robot eps1.51exfiltrati0n,14,\nSortie : \n 16 août 2016\...,"[iOS, Android]","[(0 avis), (1 avis)]","[13, 0]","[0, 0]"
1908,FIFA 17,16,\nSortie : \n 29 sept. 2016...,"[PC, PS4, ONE, PS3, 360]","[(208 avis), (209 avis), (72 avis), (8 avis), ...","[8.4, 10, 12, 7.4, 8.7]","[0, 0, 0, 0, 0]"
1909,NBA 2K17,17,\nSortie : \n 16 sept. 2016...,"[PC, PS4, ONE, PS3, 360, iOS, Android]","[(32 avis), (27 avis), (13 avis), (1 avis), (0...","[13, 15.3, 14.5, 14, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0]"
1910,Obduction,13,\nSortie : \n 24 août 2016\...,"[PC, PS4, ONE, Mac]","[(13 avis), (1 avis), (0 avis), (0 avis)]","[14.9, 16, 0, 0]","[0, 0, 0, 0]"


In [11]:
#Création de 3 listes sur la base des colonnes nombre d'avis, des notes joueurs dans le désordre 
#et de la dernière colonne avec le bon nombre d'items avis/notes mais que des valeurs à zéros 
list1=list(df1["clean_reviews"])
list2=list(df1["clean_players_rates"])
list3=list(df1["players_rates_ordered"])

#Boucle qui parcours les listes du nombre d'avis et des notes joueurs et qui vient mettre dans l'ordre les notes joueurs
#avec un 0 si le nombre d'avis est à 0 ou avec la note correspondant aux nombre d'avis sinon
for i in range(len(list1)):
    index=0
    for j in range(len(list1[i])):
            if "0" in list1[i][j]:
                list3[i][j]=0
            else:
                list3[i][j]=list2[i][index]
                index+=1

df1["players_rates_ordered"]=list3
df1.pop("clean_players_rates")
df1

,names,journalist_rate,launch_date,clean_plateforms,clean_reviews,players_rates_ordered
0,WWE 2K22,15,\nSortie : \n 11 mars 2022\...,"[PC, PS5, Xbox Series, PS4, ONE]","[(4 avis), (0 avis), (0 avis), (0 avis), (0 av...","[15, 0, 0, 0, 0]"
1,FAR : Changing Tides,14,\nSortie : \n 01 mars 2022\...,"[PC, PS5, Xbox Series, Switch, PS4, ONE]","[(3 avis), (0 avis), (1 avis), (0 avis), (0 av...","[17, 0, 16, 0, 0, 0]"
2,Babylon's Fall,12,\nSortie : \n 03 mars 2022\...,"[PC, PS5, PS4]","[(0 avis), (1 avis), (3 avis)]","[0, 0, 9]"
3,Final Fantasy Pixel Remaster,18,\nSortie : \n 29 juil. 2021...,"[PC, iOS, Android]","[(10 avis), (1 avis), (1 avis)]","[0, 6.3, 10]"
4,Assassin's Creed Valhalla : Dawn of Ragnarök,15,\nSortie : \n 10 mars 2022\...,"[PC, PS5, Xbox Series, PS4, ONE, Stadia]","[(3 avis), (1 avis), (0 avis), (0 avis), (0 av...","[12, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...
1907,Mr. Robot eps1.51exfiltrati0n,14,\nSortie : \n 16 août 2016\...,"[iOS, Android]","[(0 avis), (1 avis)]","[0, 13]"
1908,FIFA 17,16,\nSortie : \n 29 sept. 2016...,"[PC, PS4, ONE, PS3, 360]","[(208 avis), (209 avis), (72 avis), (8 avis), ...","[0, 0, 8.4, 10, 12]"
1909,NBA 2K17,17,\nSortie : \n 16 sept. 2016...,"[PC, PS4, ONE, PS3, 360, iOS, Android]","[(32 avis), (27 avis), (13 avis), (1 avis), (0...","[13, 15.3, 14.5, 14, 0, 0, 0]"
1910,Obduction,13,\nSortie : \n 24 août 2016\...,"[PC, PS4, ONE, Mac]","[(13 avis), (1 avis), (0 avis), (0 avis)]","[14.9, 16, 0, 0]"


# Explode du dataset pour avoir une entrée Jeu/Plateforme

In [12]:
#Eclatement des listes de plateformes, du nombre d'avis et de notes joueurs sur différentes lignes
#pour garder des lignes avec une entrée jeu/plateforme, un nombre d'avis et une note sur la combinaison jeu/plateforme
data_jv_final=df1.set_index(['names',"journalist_rate","launch_date"]).apply(pd.Series.explode).reset_index()
data_jv_final

,names,journalist_rate,launch_date,clean_plateforms,clean_reviews,players_rates_ordered
0,WWE 2K22,15,\nSortie : \n 11 mars 2022\...,PC,(4 avis),15
1,WWE 2K22,15,\nSortie : \n 11 mars 2022\...,PS5,(0 avis),0
2,WWE 2K22,15,\nSortie : \n 11 mars 2022\...,Xbox Series,(0 avis),0
3,WWE 2K22,15,\nSortie : \n 11 mars 2022\...,PS4,(0 avis),0
4,WWE 2K22,15,\nSortie : \n 11 mars 2022\...,ONE,(0 avis),0
...,...,...,...,...,...,...
6624,Obduction,13,\nSortie : \n 24 août 2016\...,PS4,(1 avis),16
6625,Obduction,13,\nSortie : \n 24 août 2016\...,ONE,(0 avis),0
6626,Obduction,13,\nSortie : \n 24 août 2016\...,Mac,(0 avis),0
6627,Hatsune Miku : Project Diva X,14,\nSortie : \n 30 août 2016\...,PS4,(3 avis),13


# Création d'une colonne liée à l'année de lancement du jeu

In [13]:
#Création d'une colonne avec uniquement l'année de lancement
data_jv_final["year"]=data_jv_final["launch_date"].apply(lambda x: x[-4:])
data_jv_final

,names,journalist_rate,launch_date,clean_plateforms,clean_reviews,players_rates_ordered,year
0,WWE 2K22,15,\nSortie : \n 11 mars 2022\...,PC,(4 avis),15,
1,WWE 2K22,15,\nSortie : \n 11 mars 2022\...,PS5,(0 avis),0,
2,WWE 2K22,15,\nSortie : \n 11 mars 2022\...,Xbox Series,(0 avis),0,
3,WWE 2K22,15,\nSortie : \n 11 mars 2022\...,PS4,(0 avis),0,
4,WWE 2K22,15,\nSortie : \n 11 mars 2022\...,ONE,(0 avis),0,
...,...,...,...,...,...,...,...
6624,Obduction,13,\nSortie : \n 24 août 2016\...,PS4,(1 avis),16,
6625,Obduction,13,\nSortie : \n 24 août 2016\...,ONE,(0 avis),0,
6626,Obduction,13,\nSortie : \n 24 août 2016\...,Mac,(0 avis),0,
6627,Hatsune Miku : Project Diva X,14,\nSortie : \n 30 août 2016\...,PS4,(3 avis),13,


# Filtrage des jeux ne contenant pas le nom de la plateforme dans le nom du jeu

In [14]:
#Filtre du dataset pour exclure les jeux qui comportent le nom de la plateforme dans le nom récupéré 
#et qui ne pourront pas être matchés avec le dataset initial. Exemple : "Pokémon sur Switch" ou "Animal Crossing sur PC"
data_jv_final=data_jv_final[data_jv_final["names"].str.contains(" sur ")==False]
data_jv_final

,names,journalist_rate,launch_date,clean_plateforms,clean_reviews,players_rates_ordered,year
0,WWE 2K22,15,\nSortie : \n 11 mars 2022\...,PC,(4 avis),15,
1,WWE 2K22,15,\nSortie : \n 11 mars 2022\...,PS5,(0 avis),0,
2,WWE 2K22,15,\nSortie : \n 11 mars 2022\...,Xbox Series,(0 avis),0,
3,WWE 2K22,15,\nSortie : \n 11 mars 2022\...,PS4,(0 avis),0,
4,WWE 2K22,15,\nSortie : \n 11 mars 2022\...,ONE,(0 avis),0,
...,...,...,...,...,...,...,...
6624,Obduction,13,\nSortie : \n 24 août 2016\...,PS4,(1 avis),16,
6625,Obduction,13,\nSortie : \n 24 août 2016\...,ONE,(0 avis),0,
6626,Obduction,13,\nSortie : \n 24 août 2016\...,Mac,(0 avis),0,
6627,Hatsune Miku : Project Diva X,14,\nSortie : \n 30 août 2016\...,PS4,(3 avis),13,


# Export du dataset final

In [ ]:
data_jv_final.to_csv("../data/01_GameSpy_Scrapping_jv.csv",sep=',')